# IOGS : Reconnaissance des formes - TP Deep Learning
# PARTIE 1 : Classification et finetuning

## NOM Prénom 1 - NOM Prénom 2

Pour des raisons de temps de calcul et de puissance nécessaire, il n'est pas envisageable d'optimiser un réseau de neurones de grande taille dans le cadre de ce cours. C'est pourquoi ce TP se focalise sur l'utilisation de réseaux de neurones pré-entraînés.

## Classification

### Classification sur CIFAR 100
CIFAR 100 est une base de données d'images 32x32 pour la classification. Il comporte 100 classes d'images.
Dans cette partie l'objectif est de charger un réseau et de calculer le score de classification du réseau.

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import keras
from keras.datasets import cifar10, cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import os


In [ ]:
# Download if needed the datasets
_,_ = cifar10.load_data()
_,_ = cifar100.load_data()

Nous allons maintenant définir le réseau. Le réseau choisi est basé sur VGG, c'est un type de réseau très simple mais très efficace qui utilise une suite de convolutions, normalisation, ReLU et MaxPooling.

In [ ]:
cifar100_num_classes = 100
# define the model
# INPUT SIZE (32,32,3)
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=[32,32,3]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(cifar100_num_classes))
model.add(Activation('softmax'))

Nous allons maintenant charger les poids des réseaux pré-entrainés.

In [ ]:
save_dir = "keras_models"
model_name = "cifar100_model"
model.load_weights(filepath=os.path.join(save_dir, model_name))

Il faut maintenant préparer les données pour qu'elles soient chargées et données au réseau.

In [ ]:
# data loading
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, cifar100_num_classes)
y_test = keras.utils.to_categorical(y_test, cifar100_num_classes)

# set float type and scale between 0 and 1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


Nous arrivons finalement à la partie effective du code, celle dans laquelle nous allons tester notre modèle.

<div class="alert alert-success">
<b>QUESTION : Quelle est la précision du modèle sur CIFAR 100 ?</b>
<br/>
<b>QUESTION : Quelle est la précision par classe du modèle sur CIFAR 100 ?</b>
<br/>
<b>QUESTION : Quelle est la précision moyene par classe du modèle sur CIFAR 100 (c'est la moyenne des précisiond de la question précédente)?</b>
<br/>

On pourra s'aider de la ligne de code suivante `predicted_label = np.argmax(model.predict(np.expand_dims(x_test[0], axis=0)))` qui prédit le label pour l'image 0 du jeu de test.
</div>

In [ ]:
# HERE GOES THE CODE
# ...
acc_per_class = [0 for i in range(cifar100_num_classes)]
acc = 0
acc_mean = 0
print("Accuracy",acc)
print("Accuracy per class",acc_per_class)
print("Mean accuracy", acc_mean)

### Adaptation pour CIFAR 10

CIFAR 10 est un dataset plus ancien, sur le même format que CIFAR 100 mais avec seulement 10 classes. 
Une méthode pour faire de la classification sur ces dix classes est de réentraîner un nouveau réseau.

Nous nous proposons ici d'utiliser deux approches différentes, très utilisées dans le cas où la puissance de calcul est limitée ou lorsque le dataset d'entraînement est petit, c'est à dire pas assez grand pour entraîner un réseau de neurone de cette taille.

L'idée générale est que les caractéristiques produites par le réseau sont bonnes, et que seul le classifieur néessite d'être modifié. Pour cela nous enlevons les deux dernières couches du réseau.


In [ ]:
# remove the two last layers
model.layers.pop()
model.layers.pop()
model.layers[-1].outbound_nodes = []
model.outputs = [model.layers[-1].output]
print(model.summary())

Nous devons ensuite préparer le nouveau dataset : CIFAR10.

<div class="alert alert-success">
<b>QUESTION : Charger et normaliser les données de CIFAR10.</b>
<br/>
On utilisera le code précédent et la commande de chargement `cifar10.load_data()`
</div>

In [ ]:
cifar10_num_classes = 10

...

#### Utilisation des features + SVM

La première approche consiste à récupérer les features et à entrainer un SVM sur ces features.



<div class="alert alert-success">
<b>QUESTION : Remplir un tableau numpy avec 1000 features (choisir les données au hasard) du dataset d'entraînement CIFAR10</b>
<br/>
<b>QUESTION : Entraîner un SVC de Scikit learn sur ces données</b>
<br/>
<b>QUESTION : Donner la précision, la précision par classe et la précision moyenne par classe du modèle</b>
</div>

In [ ]:
from sklearn.svm import SVC
...

##### Finetunning

Le finetuning consiste en l'utilisation d'un réseau pré-entraîné, dont on remplace les dernière couches (pour correspondre au nouveau nombre de classes) et qu'on ré-entraîne afin de spécialiser les couches ajoutées.



<div class="alert alert-success">
<b>QUESTION : Ajouter des couches au réseaux qu'il puisse faire de la classification à 10 classes.</b>
<br/>
En suivant, l'exemple du modèle précédent nous, devons ajouter deux couches.
</div>

In [ ]:
cifar10_num_classes = 10
# HERE GOES THE CODE
# ...

Étant donné que nous disposons d'une puissance de calcul limitée, nous ne voulons entraîner que la dernière couche. Pour se faire nous interdisons la mise à jour des poids pour toutes couches sauf celles que nous venons d'ajouter :

In [ ]:
for layer in model.layers[:len(model.layers)-2]:
    layer.trainable = False

Définition de l'optimiseur.
<div class="alert alert-success">
<b>QUESTION : Rappeler le principe de l'optimiseur Adam.</b>
<br/>
</div>

<div class="alert alert-warning">
<b>REPONSE</b>
<br/>

...

</div>

Définition de l'optimiseur.


In [ ]:
optimizer = keras.optimizers.Adam(lr=1e-4)

Pour entraîner le model avec la bibliothèque Keras, il faut précompiler le modèle.

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

Entraînement.

In [ ]:
epochs = 1
batch_size = 10
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

<div class="alert alert-success">
<b>QUESTION : Donner la précision, la précision par classe et la précision moyenne par classe du modèle</b>
</div>

In [ ]:
model.predict(np.expand_dims(x_train[0], axis=0))